# Comparing normal matrix results vs. multiprocessed matrix results

Making sure the multiprocessing spits out the correct results. Verified on the LUVOIR case.

In [ ]:
# Imports
import os
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import hcipy

import pastis.util
from pastis.config import CONFIG_PASTIS
from pastis.simulators.luvoir_imaging import LuvoirAPLC
datadir = CONFIG_PASTIS.get('local', 'local_data_path')

In [ ]:
# Define directories
dir_standard = '2020-07-25T17-51-31_luvoir-small'
dir_multi = '2020-07-27T20-00-29_luvoir-small'

# Define matrix name
matrix_name = 'PASTISmatrix_num_piston_Noll1.fits'

The multiprocessed case saves individual fits images, while the standard way saves out a PSF cube. It can also save out individual PSF fits images by setting the according parameter, by that's not really necessary.

In [ ]:
# Read PSF from cube in standard dir
cube_standard = fits.getdata(os.path.join(datadir, dir_standard, 'matrix_numerical', 'psfs', 'psf_cube.fits'))

Define a segment pair you want to compare the DH image for and read both PSFs.

In [ ]:
# Pick segment pair
seg1 = 46
seg2 = 78

# Read PSFs
psf_name_multi = f'psf_piston_Noll1_segs_{seg1}-{seg2}.fits'
psf_mult = fits.getdata(os.path.join(datadir, dir_multi, 'matrix_numerical', 'psfs', psf_name_multi))

standard_index = (seg1-1)*120+(seg2-1)
psf_stand = cube_standard[standard_index]

Plot both PSFs side by side to compare them. In the following cell, plot their difference - it should be zero.

In [ ]:
# Plot both
plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.title('standard')
plt.imshow(psf_stand, norm=LogNorm())
plt.subplot(1, 2, 2)
plt.title('multi')
plt.imshow(psf_mult, norm=LogNorm())

In [ ]:
# Plot difference
print(np.sum(psf_stand - psf_mult))
plt.imshow(np.log10(psf_stand - psf_mult))
plt.colorbar()

Prepare to measure their DH mean contrast by making DH mask.

In [ ]:
# Create Luvoir and DH mask
optics_input = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
sampling = 4
design = 'small'
luvoir = LuvoirAPLC(optics_input, design, sampling)   

In [ ]:
dh_outer = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

hcipy.imshow_field(dh_mask)

Compare the mean contrast in both images.

In [ ]:
# Measure contrast in both images
dh_int_stand = (psf_stand) * dh_mask.shaped
c_stand = np.mean(dh_int_stand[np.where(dh_mask.shaped != 0)])
print('contrast floor: {}'.format(c_stand))

dh_int_multi = (psf_mult) * dh_mask.shaped
c_multi = np.mean(dh_int_multi[np.where(dh_mask.shaped != 0)])
print('contrast floor: {}'.format(c_multi))

Now let's do the same on the PASTIS matrices directly. Load them, plot them side by side and then their difference.

In [ ]:
# Load both matrices
mat_stand = fits.getdata(os.path.join(datadir, dir_standard, 'matrix_numerical', matrix_name))
mat_multi = fits.getdata(os.path.join(datadir, dir_multi, 'matrix_numerical', matrix_name))

In [ ]:
# Plot both matrices
plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.imshow(mat_stand)#, norm=LogNorm())
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(mat_multi)#, norm=LogNorm())
plt.colorbar()

In [ ]:
# Plot their difference
print(np.sum(mat_stand - mat_multi))
plt.imshow(np.log10(mat_stand - mat_multi))
plt.colorbar()